<a href="https://colab.research.google.com/github/engwandrade/Bot_Linkedin_FutureTEC.py/blob/main/Cripto_Crew_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
!pip install --q langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.6/339.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [28]:
import os
from crewai_tools import tool
from crewai import Agent, Task, Crew, Process
from langchain_groq import ChatGroq

In [29]:
def final_task_callback(result):
    print("Tarefa finalizada com sucesso!")
    print("Resultado:", result)

In [30]:
import os
from google.colab import userdata

# Set the OpenAI API key from an environment variable
#os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

###**TOOLS**

In [31]:
llama3 = ChatGroq(
            api_key="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",
            model="llama3-70b-8192"
        )

In [32]:
from crewai_tools import DirectoryReadTool, ScrapeElementFromWebsiteTool, ScrapeWebsiteTool, SerperDevTool, FileReadTool, WebsiteSearchTool

docs_tool = DirectoryReadTool(directory='/content/report.md')
file_tool = FileReadTool()
search_tool = SerperDevTool()
#web_rag_tool = WebsiteSearchTool()
scrape_element_tool = ScrapeElementFromWebsiteTool()

###**AGENTS**

In [33]:
# Create agents
researcher = Agent(
    role='Especialista em pesquisas avançadas do mercado de criptomoedas',
    goal='Realiza pesquisas avançadas na internet e resume os pontos mais relevantes sobre determinado assunto.',
    backstory='Um analista especialista em buscar na internet o histórico de preços das criptomoedas bitcoin e ether',
    tools=[search_tool, scrape_element_tool],  # Adicione a nova ferramenta aqui
    llm=llama3,
    verbose=True,
    memory=True
)

investidor = Agent(
    role='Investidor especialista',
    goal='Assumir o perfil comportamental, psicográfico e demográfico de um investidor habilidoso habituado ao mercado',
    backstory='Usa informações de pesquisa para ajudar o analista de histórico com insights relevantes.',
    llm=llama3,
    verbose=True,
    memory=True
)

analista = Agent(
    role='Especialista em análise dos dados do mercado de criptomoedas e em análises de gráficos de candlestick (vela)',
    goal='Analisar os dados gerados pelo researcher detalhadamente e conversar com o investidor para estipular se serão comprados ou vendidos ativos.',
    backstory='Responsável pela análise profunda dos dados gerados pelo researcher e por conversar com o investidor.',
    llm=llama3,
    verbose=True,
    memory=True
)

analista_senior = Agent(
    role='Analista Sênior',
    goal='Cria um resumo detalhado das informações coletadas pelos outros agentes, gera insights relevantes e define se devem ser comprados ou vendidos ativos.',
    backstory='Analista do mercado de criptomoedas, especializado em Bitcoin e Ethereum, com 15 anos de experiência.',
    tools=[docs_tool, file_tool],
    llm=llama3,
    verbose=True,
    memory=True
)


###**TASKS**

In [34]:
# Define tasks with improved descriptions and focus for the researcher
research = Task(
    description='Pesquise em sites como https://br.investing.com/crypto/bitcoin/historical-data e https://br.investing.com/crypto/ethereum/historical-data.',
    expected_output='Um resumo sobre a evolução dos preços do Bitcoin e do Ethereum e insights gerados pela pesquisa.',
    agent=researcher
)

assume_investidor = Task(
    description='Assumir o papel de um investidor habilidoso do mercado de criptomoedas com anos de experiência.',
    expected_output='Fornecer conselhos e insights sobre o mercado de criptomoedas, técnicas de análise e fundamentos de investimento.',
    agent=investidor
)

analyze_investidor = Task(
    description='Analisar os dados da evolução de preço e do mercado de criptomoedas, identificando possíveis tendências.',
    expected_output='Um relatório resumido sobre tendências de alta ou de queda das criptomoedas e sobre o mercado de investimento.',
    agent=analista
)

write = Task(
    description='Escreva um resumo sobre a evolução dos preços do Bitcoin e Etherium e diga se é mais provável acontecer alta ou queda.',
    expected_output='Um texto contendo informações relevantes sobre o histórico de preços do Bitcoin e do Ethereum e suas possíveis tendências.',
    agent=analista_senior,
    output_file='/content/report.md',
    callback=final_task_callback
)

###**CREW**

In [35]:
crew = Crew(
    agents=[researcher, investidor, analista, analista_senior],
    tasks=[research, assume_investidor, analyze_investidor, write],
    manager_llm=llama3,
    full_output=True,
    verbose=2
)

crew.kickoff()

# Reabrir e salvar o arquivo com a codificação UTF-8
output_file_path = '/content/report.md'
try:
    with open(output_file_path, 'r', encoding='cp1252') as file:
        content = file.read()
    with open(output_file_path, 'w', encoding='utf-8') as file:
        file.write(content)
except UnicodeDecodeError:
    print("Erro ao decodificar o arquivo, verifique os caracteres utilizados.")


 [DEBUG]: == Working Agent: Especialista em pesquisas avançadas do mercado de criptomoedas
 [INFO]: == Starting Task: Pesquise em sites como https://br.investing.com/crypto/bitcoin/historical-data e https://br.investing.com/crypto/ethereum/historical-data.


> Entering new CrewAgentExecutor chain...
Thought: I need to gather historical price data for Bitcoin and Ethereum from reliable sources.

Action: Search the internet
Action Input: {'search_query': 'bitcoin historical price data'} 

{'message': 'Unauthorized.', 'statusCode': 403}

Thought: It seems like I got an error message from the search query. I'll try to refine my search query to get more specific results.

Action: Search the internet
Action Input: {'search_query': 'bitcoin historical price data investing.com'} 

{'message': 'Unauthorized.', 'statusCode': 403}

Thought: It seems like I'm still getting an error message. I'll try to read the website content directly from investing.com.

Action: Read a website content
Action Inp

KeyboardInterrupt: 

In [ ]:
# # Função para iniciar a missão
# def kickoff():
#     result = alianca_rebelde.kickoff()
#     # Verifique se a tarefa final foi concluída com sucesso
#     if result == "Estrela da Morte destruída, missão bem-sucedida.":
#         print("Missão concluída com sucesso!")
#         print("VENCEMOS JUNTOS!")
#         return result
#     else:
#         print("Falha na missão. Tentando novamente...")
#         return kickoff()  # Recursivamente chame kickoff até concluir com sucesso

# # Exemplo de execução da equipe
# resultado = kickoff()
# print(resultado)